In [9]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [10]:
# https://www.kaggle.com/code/goldenlock/ai4code-base?scriptVersionId=101148262
# 9025

In [11]:
root = '../working/offline/6/0'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-4.pooling_mask-attention_mask.grad_acc-4'
# pmodel2_name = 'deberta-v3-small.flag-pairwise14-2-cat.eval'

In [12]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [13]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')

In [14]:
ids = set(xc['id'])

In [15]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [16]:
calc_metric(xc, 'reg_pred')

0.9006251692558017

In [17]:
calc_metric(xc, 'pred')

0.8992880214723898

In [18]:
calc_metric(xc, 'cls_pred')

0.896173129530509

In [19]:
calc_metric(xp, 'cls_pred')

0.8966581181771137

In [20]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9044248355992179

In [21]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9018247872358709

In [22]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['pred'] * 0.5
calc_metric(x)

0.9036697563235938

In [23]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [24]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['pred'], xp['max_prob'])]
calc_metric(x)

0.908585923202479

In [25]:
df_p = pd.DataFrame(xp)

In [26]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [27]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [28]:
df_train = pd.read_feather('../working/train.fea')

In [29]:
df_train = df_train[df_train.id.isin(ids)]

In [30]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [31]:
gezi.set_fold(df, 5, 'ancestor_id')

In [32]:
df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5 = [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob = [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim = [], [], [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
for i in tqdm(range(len(df)), desc='ctop'):
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]

top:   0%|          | 0/424943 [00:00<?, ?it/s]

ctop:   0%|          | 0/424943 [00:00<?, ?it/s]

rule:   0%|          | 0/424943 [00:00<?, ?it/s]

In [33]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop2_prob', 
        ]

cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols

In [34]:
from catboost import CatBoostRegressor
import lightgbm as lgb
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          'min_data_in_leaf': 300,
          # 'max_depth': 10,
          'learning_rate': 0.02,
          "feature_fraction": 0.8,
          "bagging_fraction": 0.75,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [35]:
FOLDS = 5

In [36]:
merge_scores = []
cbt_scores, lgb_scores = [], []
scores = []
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[label_col]
  X_valid = dvalid[cols]
  y_valid = dvalid[label_col]
  cbt_model = CatBoostRegressor(**xgb_params)
  cbt_model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=500,
                )  
  # cbt_model.save_model(f'../working/trees/{fold}.cbt')
  dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
  
  x = {'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values}
  x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dvalid.pred_p.values, dvalid.pred_c.values, dvalid.max_prob.values)]
  merge_score = calc_metric(x, 'pred', df_gt)
  ic(abs(x['pred'] - dvalid['rel_rank']).mean(), merge_score)
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  lgb_model = lgb.train(params,
                  d_train,
                  10000,
                  valid_sets=[d_train, d_valid],
                  verbose_eval=500,
                  early_stopping_rounds=500)
  # lgb_model.save_model(f'../working/trees/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  merge_scores.append(merge_score)
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(merge_scores).mean(), 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )
  

  0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.2544365	test: 0.2544365	test1: 0.2543514	best: 0.2543514 (0)	total: 304ms	remaining: 50m 36s
500:	learn: 0.0632407	test: 0.0632407	test1: 0.0656280	best: 0.0656280 (500)	total: 2m 24s	remaining: 45m 37s
1000:	learn: 0.0610853	test: 0.0610853	test1: 0.0647097	best: 0.0647097 (1000)	total: 4m 36s	remaining: 41m 24s
1500:	learn: 0.0595594	test: 0.0595594	test1: 0.0644209	best: 0.0644209 (1500)	total: 6m 59s	remaining: 39m 34s
2000:	learn: 0.0583866	test: 0.0583866	test1: 0.0643415	best: 0.0643407 (1993)	total: 8m 44s	remaining: 34m 57s
2500:	learn: 0.0573591	test: 0.0573591	test1: 0.0644437	best: 0.0643313 (2133)	total: 10m 49s	remaining: 32m 26s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06433133434
bestIteration = 2133

Shrink model to first 2134 iterations.


[07/19/22 18:48:01] 3574203340.py:22 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06579407253591144
                    merge_score: 0.9070703368237418
[07/19/22 18:48:03] 3574203340.py:24 in <module>
                    cbt_score: 0.9104315790384504


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.090401 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9620
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 40
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 0.0647922	valid_1's l1: 0.0661354
[1000]	training's l1: 0.0635188	valid_1's l1: 0.0651149
[1500]	training's l1: 0.0629869	valid_1's l1: 0.0648134
[2000]	training's l1: 0.0625055	valid_1's l1: 0.0645739
[2500]	training's l1: 0.0622016	valid_1's l1: 0.064447
[3000]	training's l1: 0.0619868	valid_1's l1: 0.0643525
[3500]	training's l1: 0.0617939	valid_1's l1: 0.0642797
[4000]	training's l1: 0.0616054	valid_1's l1: 0.0642211
[4500]	training's l1: 0.0614102	valid_1's l1: 0.0641626
[5000]	training's l1: 0.0612468	valid_1's l1: 0.0641157
[5500]	training's l1: 0.0611164	va

[07/19/22 19:06:13] 3574203340.py:38 in <module>
                    lgb_score: 0.9105593900599513
[07/19/22 19:06:15] 3574203340.py:40 in <module>
                    score: 0.9108104250842369
[07/19/22 19:06:15] 3574203340.py:46 in <module>
                    fold: 0
                    np.asarray(merge_scores).mean(): 0.9070703368237418
                    np.asarray(cbt_scores).mean(): 0.9104315790384504
                    np.asarray(lgb_scores).mean(): 0.9105593900599513
                    np.asarray(scores).mean(): 0.9108104250842369


...

[07/19/22 20:14:04] 3574203340.py:38 in <module>
                    lgb_score: 0.9155733956431529
[07/19/22 20:14:05] 3574203340.py:40 in <module>
                    score: 0.9158815507917495
[07/19/22 20:14:05] 3574203340.py:46 in <module>
                    fold: 3
                    np.asarray(merge_scores).mean(): 0.9095752070572207
                    np.asarray(cbt_scores).mean(): 0.9126163234267044
                    np.asarray(lgb_scores).mean(): 0.9129131268564097
                    np.asarray(scores).mean(): 0.9131560559119043


0:	learn: 0.2544156	test: 0.2544156	test1: 0.2545067	best: 0.2545067 (0)	total: 159ms	remaining: 26m 31s
500:	learn: 0.0631691	test: 0.0631691	test1: 0.0656771	best: 0.0656771 (500)	total: 50.3s	remaining: 15m 54s
1000:	learn: 0.0610502	test: 0.0610502	test1: 0.0647680	best: 0.0647674 (999)	total: 1m 43s	remaining: 15m 28s
1500:	learn: 0.0595070	test: 0.0595070	test1: 0.0644590	best: 0.0644590 (1500)	total: 2m 35s	remaining: 14m 38s
2000:	learn: 0.0583593	test: 0.0583593	test1: 0.0643696	best: 0.0643680 (1964)	total: 3m 31s	remaining: 14m 6s
2500:	learn: 0.0573699	test: 0.0573699	test1: 0.0643467	best: 0.0643405 (2462)	total: 4m 25s	remaining: 13m 16s
3000:	learn: 0.0566059	test: 0.0566059	test1: 0.0643463	best: 0.0643402 (2567)	total: 5m 19s	remaining: 12m 25s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06434017347
bestIteration = 2567

Shrink model to first 2568 iterations.


[07/19/22 20:19:38] 3574203340.py:22 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06573988537486983
                    merge_score: 0.9039693073200067
[07/19/22 20:19:40] 3574203340.py:24 in <module>
                    cbt_score: 0.9063462862332429


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035271 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9618
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 40
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 0.0646559	valid_1's l1: 0.0661909
[1000]	training's l1: 0.0634453	valid_1's l1: 0.0652767
[1500]	training's l1: 0.0629304	valid_1's l1: 0.0649668
[2000]	training's l1: 0.0625989	valid_1's l1: 0.0647895
[2500]	training's l1: 0.062325	valid_1's l1: 0.0646627
[3000]	training's l1: 0.0620582	valid_1's l1: 0.0645413
[3500]	training's l1: 0.0618253	valid_1's l1: 0.0644568
[4000]	training's l1: 0.0616556	valid_1's l1: 0.064407
[4500]	training's l1: 0.0614767	valid_1's l1: 0.0643504
[5000]	training's l1: 0.0613315	valid_1's l1: 0.0643022
[5500]	training's l1: 0.0611098	val

[07/19/22 20:23:13] 3574203340.py:38 in <module>
                    lgb_score: 0.9067770507000141
[07/19/22 20:23:15] 3574203340.py:40 in <module>
                    score: 0.9069607880602689
[07/19/22 20:23:15] 3574203340.py:46 in <module>
                    fold: 4
                    np.asarray(merge_scores).mean(): 0.908454027109778
                    np.asarray(cbt_scores).mean(): 0.911362315988012
                    np.asarray(lgb_scores).mean(): 0.9116859116251306
                    np.asarray(scores).mean(): 0.9119170023415772


In [37]:
gezi.plot.feature_importance(cbt_model, topn=20)

UnboundLocalError: local variable 'feat_importances' referenced before assignment

In [ ]:
gezi.plot.feature_importance(lgb_model, topn=20)